In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../data/processed/baseline/train/2017-01-01.csv', index_col=0)
df.head()

,B04_width,B04_height,B04_size,B04_mean,B04_std,B10_width,B10_height,B10_size,B10_mean,B10_std,...,TCI_height,TCI_size,TCI_mean,TCI_std,B09_width,B09_height,B09_size,B09_mean,B09_std,Crop_Id_Ne
Field_Id,,,,,,,,,,,,,,,,,,,,,
19,20,13,260,881.438462,463.475964,4,3,12,6.083333,6.102709,...,13,260,63.302564,32.484367,4,3,12,315.250000,315.312576,8
20,11,23,253,625.750988,369.445816,3,5,15,2.466667,4.951319,...,23,253,50.411067,29.792071,3,5,15,129.133333,258.393722,8
21,21,6,126,416.325397,475.222702,4,2,8,1.625000,4.299346,...,6,126,31.550265,35.629915,4,2,8,69.000000,182.556840,8
22,23,14,322,880.639752,466.805984,5,3,15,5.933333,5.603174,...,14,322,62.554865,32.949057,5,3,15,313.666667,294.116001,8
24,8,28,224,596.223214,349.765156,3,5,15,3.533333,5.031457,...,28,224,49.212798,28.520267,3,5,15,221.666667,314.196046,8


In [85]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, log_loss

seed=2019

In [15]:
y = df.Crop_Id_Ne
X = df.drop('Crop_Id_Ne', axis=1)

log_loss_scorer = make_scorer(log_loss)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25, random_state=seed)

tpot = TPOTClassifier(
    generations=5, 
    population_size=20, 
    verbosity=2, 
    n_jobs=-1,  
    scoring='accuracy',
    cv=2
)
tpot.fit(X_train, y_train)

Generation 1 - Current best internal CV score: 0.6734732705867735
Generation 2 - Current best internal CV score: 0.6734732705867735
Generation 3 - Current best internal CV score: 0.6734732705867735
Generation 4 - Current best internal CV score: 0.6757641412729476
Generation 5 - Current best internal CV score: 0.6763193862019694

Best pipeline: LogisticRegression(ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.3, min_samples_leaf=14, min_samples_split=11, n_estimators=100), C=10.0, dual=False, penalty=l2)


TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=2,
        disable_update_check=False, early_stop=None, generations=5,
        max_eval_time_mins=5, max_time_mins=None, memory=None,
        mutation_rate=0.9, n_jobs=-1, offspring_size=None,
        periodic_checkpoint_folder=None, population_size=20,
        random_state=None, scoring='accuracy', subsample=1.0,
        template='RandomTree', use_dask=False, verbosity=2,
        warm_start=False)

In [16]:
print(tpot.score(X_test, y_test))

0.6996587030716723


In [17]:
tpot.export('tpot_pipeline.py')

In [32]:
test_data = pd.read_csv('../../data/processed/baseline/test/2017-01-01.csv', index_col=0)
test_data.head()

,B04_width,B04_height,B04_size,B04_mean,B04_std,B10_width,B10_height,B10_size,B10_mean,B10_std,...,TCI_width,TCI_height,TCI_size,TCI_mean,TCI_std,B09_width,B09_height,B09_size,B09_mean,B09_std
Field_Id,,,,,,,,,,,,,,,,,,,,,
5,10,10,100,1.000000,0.000000,10,10,100,1.000000,0.000000,...,10,10,100,1.000000,0.000000,10,10,100,1.000000,0.000000
6,10,10,100,1.000000,0.000000,10,10,100,1.000000,0.000000,...,10,10,100,1.000000,0.000000,10,10,100,1.000000,0.000000
10,10,10,100,1.000000,0.000000,10,10,100,1.000000,0.000000,...,10,10,100,1.000000,0.000000,10,10,100,1.000000,0.000000
18,28,10,280,677.260714,462.031059,5,3,15,3.733333,5.284779,...,28,10,280,52.651190,33.494008,5,3,15,216.733333,307.921953
23,23,17,391,638.281330,281.067271,4,3,12,8.916667,2.752524,...,23,17,391,53.161978,23.495900,4,3,12,553.333333,167.755940


In [33]:
test_sub = pd.read_csv('../../data/raw/sample_submission_fixed.csv', index_col=0)
test_sub.head()

,crop_id_1,crop_id_2,crop_id_3,crop_id_4,crop_id_5,crop_id_6,crop_id_7,crop_id_8,crop_id_9
field_id,,,,,,,,,
5,0.00321,0.4321,0.677,0.1323,0.97,0.432,0.432,0.123,0.432
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
probs = tpot.predict_proba(test_data)
probs_df = pd.DataFrame(probs, columns=test_sub.columns)
probs_df['Field_Id'] = test_data.index.values
probs_df.set_index('Field_Id', inplace=True)
probs_df.head()

,crop_id_1,crop_id_2,crop_id_3,crop_id_4,crop_id_5,crop_id_6,crop_id_7,crop_id_8,crop_id_9
Field_Id,,,,,,,,,
5,0.005963,2.537193e-01,0.253685,0.009123,0.119354,0.001434,0.000007,0.103008,0.253707
6,0.005963,2.537193e-01,0.253685,0.009123,0.119354,0.001434,0.000007,0.103008,0.253707
10,0.005963,2.537193e-01,0.253685,0.009123,0.119354,0.001434,0.000007,0.103008,0.253707
18,0.000683,2.573541e-09,0.020121,0.013407,0.008268,0.033456,0.094682,0.727938,0.101445
23,0.000069,1.854944e-12,0.002282,0.128150,0.000045,0.035069,0.279402,0.462156,0.092828


In [35]:
probs_df.to_csv('../../data/submissions/baseline_tpot.csv')